# Import Packages

In [1]:
import pandas as pd
import language_tool_python
import numpy as np
import re
from tqdm import tqdm
from loguru import logger
from fuzzywuzzy import fuzz
from gingerit.gingerit import GingerIt

In [2]:
tqdm.pandas()

# Loading data

In [13]:
data = pd.read_csv('DataStore/ScrapedData_pg_v1.csv')
special_char = pd.read_csv('Special characters list.csv')

In [17]:
data['product_bullets'][1]

'COMFORTABLE : When babies are swaddled, they sleep better & feel safe as babies are used to being wrapped tightly in the womb. Bamboo Fabric is weightless & thermo-regulating which helps to regulate the body temperature based on the weather.,SOFTER THAN SOFT : The bamboo fabric is super-soft and gentle on your baby’s skin. The soft plush fabric wraps around your baby for a comfortable sleep. It is also the best choice for rash prevention.,MULTIPURPOSE : Mush Bamboo Swaddles can be used as a play mat, a blanket, a burp cloth, a nursing cover, a picnic blanket. It is one of the best baby shower gifts for every new mom.'

In [4]:
data.fillna('NULL',inplace = True)

In [5]:
# my_tool = language_tool_python.LanguageTool('en-US')
my_tool = GingerIt()

In [20]:
def runGinger(txt,my_tool):
    sentences = txt.split('. ')
    flg = 1
    corrections = []
    for sentence in sentences:
        if len(sentence)>280:
            words = sentence.split(' ')
            for i in range(len(words)-50):
                sen = ' '.join(words[i:i+50])
                parse_res = my_tool.parse(sen)
                if len(parse_res['corrections'])>0:
                    flg = 0
                corrections.append(parse_res['corrections'])
        else:
            parse_res = my_tool.parse(sentence)
            if len(parse_res['corrections'])>0:
                flg = 0
            corrections.append(parse_res['corrections'])
    return [flg,corrections]
        
    


In [21]:

data['product_bullets'].apply(lambda x: runGinger(x,my_tool))

KeyboardInterrupt: 

In [7]:
data['bullet_points'][0].split('\n')

['Large Size - 75 cms X 150 cms ; Material : Terry - 100% Bamboo Overall: 70% Bamboo 30% Cotton; Weight: 600 GSM',
 'Highly Absorbent and Odorless : Bamboo Fibers are 3 times more absorbent than Cotton due to its porous cross section',
 'Ultra soft: bamboo Towels are softer than softest cotton with a natural sheen like silk and cashmere']

In [28]:
res = my_tool.parse(data['bullet_points'][0])

In [29]:
res

{'text': 'Large Size - 75 cms X 150 cms ; Material : Terry - 100% Bamboo Overall: 70% Bamboo 30% Cotton; Weight: 600 GSM\nHighly Absorbent and Odorless : Bamboo Fibers are 3 times more absorbent than Cotton due to its porous cross section\nUltra soft: bamboo Towels are softer than softest cotton with a natural sheen like silk and cashmere',
 'result': 'Large Size - 75 cms X 150 cms; Material: Terry - 100% Bamboo Overall: 70% Bamboo 30% Cotton; Weight: 600 GSM\nHighly Absorbent and Odorless: Bamboo Fibers are 3 times more absorbent than Cotton due to its porous cross section\nUltra soft: bamboo Towels are softer than the softest cotton with a natural sheen like silk and cashmere',
 'corrections': [{'start': 270,
   'text': 'softest cotton',
   'correct': 'the softest cotton',
   'definition': None},
  {'start': 140, 'text': ' :', 'correct': ':', 'definition': 'Accept space'},
  {'start': 40, 'text': ' :', 'correct': ':', 'definition': 'Accept space'},
  {'start': 29, 'text': ' ;', 'corr

# Helper Functions

## Sentence Case

In [10]:
def sentence_case(brand_name,title,brand_present_title):
    first_word = title.strip().split(' ')[0]
#     print(brand_name,first_word,brand_present_title)
    if brand_present_title==1:
        if brand_name.strip()[0]==first_word[0]:
            res = 1
        else:
            res = 0
    else:
        if first_word[0].isupper():
            res = 1
        else:
            res = 0
#     print(res)
    return res

## Spell Check

In [11]:
def spellcheck(my_text,my_tool):
    # using the tool
#     logger.info(my_text)  
    # getting the matches  
    my_matches = my_tool.check(my_text)  
#     logger.info(my_matches)
    # defining some variables  
    myMistakes = []  
    myCorrections = []  
    startPositions = []  
    endPositions = []  

    # using the for-loop  
    for rules in my_matches:  
        if len(rules.replacements) > 0:  
#             startPositions.append(rules.offset)  
#             endPositions.append(rules.errorLength + rules.offset)  
            myMistakes.append(my_text[rules.offset : rules.errorLength + rules.offset])  
            myCorrections.append(rules.replacements[0])
    if len(myMistakes)>0:
        return 0
    else:
        return 1
    return [myMistakes,myCorrections,startPositions,endPositions]
    

## Special Character Check

In [29]:
# len(set(special_char[0].tolist())-set("pr"+chr(9786)+"atik"))

# len(set(special_char[0]))

126

In [12]:
def special_char_check(x):
    if len(set(special_char[0])) - len(set(special_char[0].tolist())-set(x))>0:
        return 0
    else:
        return 1

# Title Check

## Brand Name Present

In [13]:
bn_check = lambda x,y:1 if y.split(' ')[0].strip().lower()==x.strip().lower() else 0

In [14]:
data['title_brand_present'] = data[['brand_name','title']].apply(lambda x:bn_check(x.brand_name,x.title),axis = 1)

## Sentence Case

In [15]:
data['title_sentence_case'] = data[['brand_name','title',"title_brand_present"]].apply(lambda x:sentence_case(x.brand_name,x.title,x.title_brand_present),axis = 1)

## Spell Check

In [16]:
data['title_spellcheck'] = data['title'].progress_apply(lambda x:spellcheck(x,my_tool))

100%|██████████| 2580/2580 [01:07<00:00, 38.48it/s]


In [13]:
data['title'][3]

'Mush 600 GSM Hand Towel Set of 2 | 100% Bamboo |Ultra Soft, Absorbent & Quick Dry Towel for Gym, Pool, Travel, Spa and Yoga | 29.5 x 14 Inches (Navy Blue & Grey)'

In [98]:
# spellcheck('Mush 600 GSM Hand Towele Seet of 2 | 100% Bamboo |Ultro Soft, Absorbent & Quick Dry Towel for Gym, Pool, Travel, Spa and Yoga | 29.5 x 14 Inches (Navy Blue & Grey)', my_tool)

[['Towele', 'Seet', 'Ultro'], ['Towel', 'Set', 'Ultra'], [], []]

## Final Title check Flag

In [ ]:
def get_Title_flag(data):
    ## Brand Name Present
    bn_check = lambda x,y:1 if y.split(' ')[0].strip().lower()==x.strip().lower() else 0
    data['title_brand_present'] = data[['brand_name','title']].apply(lambda x:bn_check(x.brand_name,x.title),axis = 1)

    ## Sentence Case
    data['title_sentence_case'] = data[['brand_name','title',"title_brand_present"]].apply(lambda x:sentence_case(x.brand_name,x.title,x.title_brand_present),axis = 1)

    ## Spell Check
    data['title_spellcheck'] = data['title'].progress_apply(lambda x:spellcheck(x,my_tool))
    
    return data[['title_brand_present','title_sentence_case','title_spellcheck']].product(axis = 1)

In [158]:
data['final_title_check_flag'] = data[['title_brand_present','title_sentence_case','title_spellcheck']].product(axis = 1)

In [159]:
data[['title_brand_present','title_sentence_case','title_spellcheck']].product(axis = 1)

0       1
1       1
2       0
3       0
4       0
       ..
2575    1
2576    0
2577    1
2578    1
2579    0
Length: 2580, dtype: int64

# Description Check

## Special Character Check

In [17]:
data['description_special_chr_check'] = data['description'].apply(lambda x:special_char_check(x))

## Characters Constrained

In [18]:
data['description_char_constrained_2000'] = data['description'].apply(lambda x:1 if len(x.strip())<=2000 else 0)

## Multiline Check

In [19]:
def multiline_check(x):
    first_str = data['description'][3]
    order = "[+-]?\d+\.\d+"
    first_str = re.sub(order, '', first_str)
    lines = re.split( r'[?.!]',first_str)
    if len(lines)>1:
        return 1
    else:
        return 0

In [32]:
print(data['description'][3])
first_str = data['description'][3]
order = "[+-]?\d+\.\d+"
first_str = re.sub(order, '', first_str)
print(first_str)
re.split( r'[?.!]',first_str)

Mush hand towels are made out of 100% Bamboo Terry which makes it extremely soft, 3x more absorbent than cotton, naturally anti-odor, and moisture wicking. Different colors for you to pick. 2 pcs 29.5" x 14" perfect size for Napkin, poolside, bathroom, Beach, sauna, college dorm room, spa or gym use. It comes in an eco friendly reusable fabric bag making it easy to carry on the go The bamboo fibers ensure quick dry, odorless properties. Switch to Bamboo today ! Why Bamboo? The fibers made out of Bamboo have numerous performance advantages as compared to others: Performance Advantage: - Ultra Soft - Highly absorbent (Up to 3 times more absorbent than cotton)
Mush hand towels are made out of 100% Bamboo Terry which makes it extremely soft, 3x more absorbent than cotton, naturally anti-odor, and moisture wicking. Different colors for you to pick. 2 pcs " x 14" perfect size for Napkin, poolside, bathroom, Beach, sauna, college dorm room, spa or gym use. It comes in an eco friendly reusable

['Mush hand towels are made out of 100% Bamboo Terry which makes it extremely soft, 3x more absorbent than cotton, naturally anti-odor, and moisture wicking',
 ' Different colors for you to pick',
 ' 2 pcs " x 14" perfect size for Napkin, poolside, bathroom, Beach, sauna, college dorm room, spa or gym use',
 ' It comes in an eco friendly reusable fabric bag making it easy to carry on the go The bamboo fibers ensure quick dry, odorless properties',
 ' Switch to Bamboo today ',
 ' Why Bamboo',
 ' The fibers made out of Bamboo have numerous performance advantages as compared to others: Performance Advantage: - Ultra Soft - Highly absorbent (Up to 3 times more absorbent than cotton)']

In [20]:
data['description_multiline_check'] = data['description'].apply(lambda x:multiline_check(x))

## Spell Check 

In [21]:
data['description_spellcheck'] = data['description'].progress_apply(lambda x:spellcheck(x,my_tool))

100%|██████████| 2580/2580 [01:03<00:00, 40.66it/s]


## Final Description check Flag

In [160]:
data['final_description_check_flag'] = data[['description_special_chr_check','description_char_constrained_2000','description_multiline_check','description_spellcheck']].product(axis = 1)

# Bullet Point Check

## Special Character check

In [22]:
data['bullets_special_chr_check'] = data['bullet_points'].apply(lambda x:special_char_check(x))

## Number of bullet points check (atleast 3 points)

In [25]:
data['bullets_number_check'] = data['bullet_points'].progress_apply(lambda x:1 if x.count('\n')>=2 else 0)

100%|██████████| 2580/2580 [00:00<00:00, 627212.91it/s]


## Bullet Points start with capital letter check

In [26]:
data['bullets_first_capital_check'] = data['bullet_points'].apply(lambda x: int(''.join([s[0] for s in x.split('\n')]).isupper()) )

## Spell Check

In [27]:
data['bullets_spellcheck'] = data['bullet_points'].progress_apply(lambda x:spellcheck(x,my_tool))

100%|██████████| 2580/2580 [01:34<00:00, 27.27it/s]


## Final Bullet Points check Flag

In [161]:
data['final_bullet_point_check_flag'] = data[['bullets_special_chr_check','bullets_number_check','bullets_first_capital_check','bullets_spellcheck']].product(axis = 1)

# Entire Spell Check

AND of title, description and bullet points spell check

In [28]:
def get_sum(lst):
    return sum(lst)

In [162]:
data['final_entire_spellcheck'] = data[['title_spellcheck','description_spellcheck','bullets_spellcheck']].apply(lambda x:1 if get_sum([x.title_spellcheck,x.description_spellcheck,x.bullets_spellcheck])==3 else 0,axis = 1)

# Dimension Details Check

In [ ]:
data['complete_data'] = data['title']+data['description']+data['bullet_points']#+

In [24]:
# data['complete_data'][0]

# # ## Regex 
# # "(((\d+ ?[a-zA-Z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?))|((\d+ ?[a-z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?)))|(((\d+ ?)[x,X] ?(\d+ ?)[x,X] ?(\d+ ?[a-zA-Z]+))|((\d+ ?)[x,X] ?(\d+ ?[a-zA-Z]+)))"

# for i in ['m','cms','mm','km','inches','ft']:
#     logger.info('{},{}'.format(i,fuzz.ratio('cms',i)))

In [133]:
def qc_dim(unit,values):
#     metric = {'meter':['m','meter'],
#     'centimeter':['cm','cms','centimeter'],
#     'millimeter':['mm','millimeter'],
#     'inches':['inch','inches']
#     'litre':['l','lit','litre'],
#     'gram':['g','gm','gram'],
#     'kilogram':['kg','kgms','kilogram']}
#     metric = ['meter','centimeter','millimeter','kilometer','inches','foot']
    metric2 = ['m','cms','mm','km','inches','ft']
    metric_change = [1,0.01,0.001,1000,0.0254,0.0348]
    metric_unit= []
    logger.info('unit for qc {}\nvalue for qc {}'.format(unit,values))
    for u in unit:
        umetric_ratio = []
        for m in metric2:
            umetric_ratio.append(fuzz.ratio(u,m))
        metric_unit.append(metric2[np.array(umetric_ratio).argmax()])
    logger.info('metric_unit {}'.format(metric_unit))
    updated_values = [float(v)*metric_change[metric2.index(i)] for i,v in zip(metric_unit,values)]
    logger.info('updated values {}'.format(updated_values))
    if len(set(metric_unit))>1:
        qc_res = [0,updated_values]
    else:
        qc_res = [1,updated_values]
    
    return qc_res

In [137]:
def format_dim(dim):
    logger.info('initial dim {}'.format(dim))
    dim = dim.replace(' ','').lower()
    dim = dim.replace('x',' ')
    units = []
    for unit in re.finditer("[a-z]+",dim):
        units.append(unit.group())
    values = []
    for val in re.finditer("[0-9]+",dim):
        values.append(val.group())
    logger.info('unit is {} and values are {}'.format(units,values))
    return [units,values]

In [149]:
def check_values(value_list):
    logger.info('value_list for check values {}'.format(value_list))
    value_list = [sorted(l) for l in value_list]
    logger.info('after sort {}'.format(value_list))
    same_val_list = np.array(value_list).T.tolist()
    logger.info('transform for each dim {}'.format(same_val_list))
    res = 1
    for v in same_val_list:
        ratio_list = [v[i]/v[0] for i in range(len(v))]
        logger.info('ratio list for list {} {}'.format(v,ratio_list))
        res*=np.prod([1 if (i<=1.05 and i>=0.95) else 0 for i in ratio_list])
    logger.info('res {}'.format(res))
    return res

In [168]:
data['complete_data'][0]

'Mush 600 GSM Bath Towel 1 - Olive Green | 100% Bamboo |Ultra Soft, Absorbent & Quick Dry Towel for Bath, Beach, Pool, Travel, Spa & Shower | 29 x 59 Inches--Large Size - 75 cms X 150 cms ; Material : Terry - 100% Bamboo Overall: 70% Bamboo 30% Cotton; Weight: 600 GSM\nHighly Absorbent and Odorless : Bamboo Fibers are 3 times more absorbent than Cotton due to its porous cross section\nUltra soft: bamboo Towels are softer than softest cotton with a natural sheen like silk and cashmere'

In [167]:
iters = re.finditer("(((\d+ ?[a-zA-Z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?))|((\d+ ?[a-z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?)))|(((\d+ ?)[x,X] ?(\d+ ?)[x,X] ?(\d+ ?[a-zA-Z]+))|((\d+ ?)[x,X] ?(\d+ ?[a-zA-Z]+)))",data['complete_data'][0])
matched_strings = []
for i in iters:
    matched_strings.append(i.group())
matched_strings

['29 x 59 Inches', '75 cms X 150 cms ']

In [134]:
def get_dimensions(text):
    iters = re.finditer("(((\d+ ?[a-zA-Z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?))|((\d+ ?[a-z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?)))|(((\d+ ?)[x,X] ?(\d+ ?)[x,X] ?(\d+ ?[a-zA-Z]+))|((\d+ ?)[x,X] ?(\d+ ?[a-zA-Z]+)))",text)
    matched_strings = []
    for i in iters:
        matched_strings.append(i.group())
    logger.info('matched_strings {}'.format(matched_strings))
    if len(matched_strings)==0:
        return [0,0,0]
    same_unit_in_dim = 1
    multi_units_value = []
    for dim in matched_strings:
        units, values = format_dim(dim)
        if len(units)==1:
            units = units*len(values)
        qc_res = qc_dim(units,values)
        same_unit_in_dim*=qc_res[0]
        multi_units_value.append(qc_res[1])
    return [1,same_unit_in_dim,check_values(multi_units_value)]
            

In [153]:
data['dimensionality_inter_check'] = data['complete_data'].progress_apply(lambda x: get_dimensions(x))

  0%|          | 0/2580 [00:00<?, ?it/s]2023-01-24 16:44:41.310 | INFO     | __main__:get_dimensions:6 - matched_strings ['29 x 59 Inches', '75 cms X 150 cms ']
2023-01-24 16:44:41.311 | INFO     | __main__:format_dim:2 - initial dim 29 x 59 Inches
2023-01-24 16:44:41.312 | INFO     | __main__:format_dim:11 - unit is ['inches'] and values are ['29', '59']
2023-01-24 16:44:41.313 | INFO     | __main__:qc_dim:13 - unit for qc ['inches', 'inches']
value for qc ['29', '59']
2023-01-24 16:44:41.314 | INFO     | __main__:qc_dim:19 - metric_unit ['inches', 'inches']
2023-01-24 16:44:41.316 | INFO     | __main__:qc_dim:21 - updated values [0.7365999999999999, 1.4986]
2023-01-24 16:44:41.316 | INFO     | __main__:format_dim:2 - initial dim 75 cms X 150 cms 
2023-01-24 16:44:41.317 | INFO     | __main__:format_dim:11 - unit is ['cms', 'cms'] and values are ['75', '150']
2023-01-24 16:44:41.318 | INFO     | __main__:qc_dim:13 - unit for qc ['cms', 'cms']
value for qc ['75', '150']
2023-01-24 16:4

2023-01-24 16:44:41.372 | INFO     | __main__:format_dim:2 - initial dim 5 X 14 InchesEnhance
2023-01-24 16:44:41.373 | INFO     | __main__:format_dim:11 - unit is ['inchesenhance'] and values are ['5', '14']
2023-01-24 16:44:41.373 | INFO     | __main__:qc_dim:13 - unit for qc ['inchesenhance', 'inchesenhance']
value for qc ['5', '14']
2023-01-24 16:44:41.374 | INFO     | __main__:qc_dim:19 - metric_unit ['inches', 'inches']
2023-01-24 16:44:41.375 | INFO     | __main__:qc_dim:21 - updated values [0.127, 0.35559999999999997]
2023-01-24 16:44:41.376 | INFO     | __main__:check_values:2 - value_list for check values [[0.127, 0.35559999999999997]]
2023-01-24 16:44:41.377 | INFO     | __main__:check_values:4 - after sort [[0.127, 0.35559999999999997]]
2023-01-24 16:44:41.378 | INFO     | __main__:check_values:6 - transform for each dim [[0.127], [0.35559999999999997]]
2023-01-24 16:44:41.380 | INFO     | __main__:check_values:10 - ratio list for list [0.127] [1.0]
2023-01-24 16:44:41.382 

2023-01-24 16:44:41.444 | INFO     | __main__:format_dim:11 - unit is ['inches'] and values are ['14', '14']
2023-01-24 16:44:41.445 | INFO     | __main__:qc_dim:13 - unit for qc ['inches', 'inches']
value for qc ['14', '14']
2023-01-24 16:44:41.447 | INFO     | __main__:qc_dim:19 - metric_unit ['inches', 'inches']
2023-01-24 16:44:41.449 | INFO     | __main__:qc_dim:21 - updated values [0.35559999999999997, 0.35559999999999997]
2023-01-24 16:44:41.450 | INFO     | __main__:check_values:2 - value_list for check values [[0.35559999999999997, 0.35559999999999997]]
2023-01-24 16:44:41.451 | INFO     | __main__:check_values:4 - after sort [[0.35559999999999997, 0.35559999999999997]]
2023-01-24 16:44:41.452 | INFO     | __main__:check_values:6 - transform for each dim [[0.35559999999999997], [0.35559999999999997]]
2023-01-24 16:44:41.453 | INFO     | __main__:check_values:10 - ratio list for list [0.35559999999999997] [1.0]
2023-01-24 16:44:41.454 | INFO     | __main__:check_values:10 - rat

2023-01-24 16:44:41.509 | INFO     | __main__:qc_dim:13 - unit for qc ['cms', 'cms']
value for qc ['35', '75']
2023-01-24 16:44:41.510 | INFO     | __main__:qc_dim:19 - metric_unit ['cms', 'cms']
2023-01-24 16:44:41.511 | INFO     | __main__:qc_dim:21 - updated values [0.35000000000000003, 0.75]
2023-01-24 16:44:41.511 | INFO     | __main__:check_values:2 - value_list for check values [[0.127, 0.35559999999999997], [0.35000000000000003, 0.75]]
2023-01-24 16:44:41.512 | INFO     | __main__:check_values:4 - after sort [[0.127, 0.35559999999999997], [0.35000000000000003, 0.75]]
2023-01-24 16:44:41.512 | INFO     | __main__:check_values:6 - transform for each dim [[0.127, 0.35000000000000003], [0.35559999999999997, 0.75]]
2023-01-24 16:44:41.513 | INFO     | __main__:check_values:10 - ratio list for list [0.127, 0.35000000000000003] [1.0, 2.755905511811024]
2023-01-24 16:44:41.514 | INFO     | __main__:check_values:10 - ratio list for list [0.35559999999999997, 0.75] [1.0, 2.10911136107986

2023-01-24 16:44:41.590 | INFO     | __main__:format_dim:2 - initial dim 35 cms X 75 cms 
2023-01-24 16:44:41.591 | INFO     | __main__:format_dim:11 - unit is ['cms', 'cms'] and values are ['35', '75']
2023-01-24 16:44:41.593 | INFO     | __main__:qc_dim:13 - unit for qc ['cms', 'cms']
value for qc ['35', '75']
2023-01-24 16:44:41.596 | INFO     | __main__:qc_dim:19 - metric_unit ['cms', 'cms']
2023-01-24 16:44:41.598 | INFO     | __main__:qc_dim:21 - updated values [0.35000000000000003, 0.75]
2023-01-24 16:44:41.599 | INFO     | __main__:check_values:2 - value_list for check values [[0.127, 0.35559999999999997], [0.35000000000000003, 0.75]]
2023-01-24 16:44:41.599 | INFO     | __main__:check_values:4 - after sort [[0.127, 0.35559999999999997], [0.35000000000000003, 0.75]]
2023-01-24 16:44:41.600 | INFO     | __main__:check_values:6 - transform for each dim [[0.127, 0.35000000000000003], [0.35559999999999997, 0.75]]
2023-01-24 16:44:41.601 | INFO     | __main__:check_values:10 - ratio

...

2023-01-24 16:44:43.257 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.257 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.257 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.258 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.258 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.259 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.259 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.259 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.260 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.261 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.262 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.268 | INFO     | __main__:get_dimensions:6 - matched_str

2023-01-24 16:44:43.322 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.323 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.323 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.324 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.324 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.325 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.325 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.325 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.326 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.326 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.327 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.327 | INFO     | __main__:get_dimensions:6 - matched_str

2023-01-24 16:44:43.379 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.380 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.381 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.382 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.384 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.384 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.385 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.385 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.386 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.386 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.387 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.387 | INFO     | __main__:get_dimensions:6 - matched_str

2023-01-24 16:44:43.455 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.455 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.456 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.457 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.458 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.459 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.459 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.460 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.461 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.462 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.463 | INFO     | __main__:get_dimensions:6 - matched_strings []
2023-01-24 16:44:43.464 | INFO     | __main__:get_dimensions:6 - matched_str

2023-01-24 16:44:43.520 | INFO     | __main__:qc_dim:21 - updated values [0.15, 0.2]
2023-01-24 16:44:43.520 | INFO     | __main__:check_values:2 - value_list for check values [[0.15, 0.2]]
2023-01-24 16:44:43.521 | INFO     | __main__:check_values:4 - after sort [[0.15, 0.2]]
2023-01-24 16:44:43.521 | INFO     | __main__:check_values:6 - transform for each dim [[0.15], [0.2]]
2023-01-24 16:44:43.522 | INFO     | __main__:check_values:10 - ratio list for list [0.15] [1.0]
2023-01-24 16:44:43.522 | INFO     | __main__:check_values:10 - ratio list for list [0.2] [1.0]
2023-01-24 16:44:43.522 | INFO     | __main__:check_values:12 - res 1
100%|██████████| 2580/2580 [00:02<00:00, 1165.87it/s]


In [155]:
data['dimensionality_check'] = data['dimensionality_inter_check'].apply(lambda x: np.prod(x))

In [156]:
data['dimensionality_check'] 

0       1
1       1
2       0
3       1
4       1
       ..
2575    0
2576    0
2577    0
2578    0
2579    1
Name: dimensionality_check, Length: 2580, dtype: int32

In [163]:
data['final_dimensionality_check'] = data['dimensionality_check']

# Sentence Case (only for Title) Check

In [164]:
data['final_sentence_case_check'] = data['title_sentence_case']

# A+ Content Check

# Save Result

In [165]:
data.to_csv('Excel_check_python_pg_v1.csv',index = False)

In [1]:
import streamlit_authenticator as stauth
import yaml
from yaml.loader import SafeLoader


In [2]:
passwords = ['first_app_pg','AIgoGLOCAL']
hashed_passwords = stauth.Hasher(passwords).generate()
hashed_passwords

['$2b$12$kcs7x1QPx5bQ3nOBmTkj7ei6CGKY54oArbW6n7TYn3MbnJMEBdt2C',
 '$2b$12$Arf1zkDyDV0Dn2ZPvG4mrOTSBAqdjFvByZYwuwttqqhL0l4qBdR6G']